# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [4]:
import pandas as pd
insurance_data = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')

In [6]:
pd.set_option('display.max_columns', None)
insurance_data.head()


,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,Suburban,Married,61,7.0,52,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,Suburban,Single,64,3.0,26,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,Suburban,Single,100,34.0,31,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,Suburban,Single,97,10.0,3,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,23675,Suburban,Married,117,NaN,31,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [ ]:
filtered_df = insurance_data[
    (insurance_data['Total Claim Amount'] > 1000) &  
    (insurance_data['Response'] == 'Yes')
]

filtered_df

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [25]:
filtered_df_2 = insurance_data[(insurance_data['Response'] == 'Yes')]
grouped = filtered_df_2.groupby(['Policy Type', 'Gender']).agg({'Total Claim Amount':['mean']}).round(2)

grouped

Total Claim Amount
                                    mean
Policy Type    Gender                   
Corporate Auto F                  433.74
               M                  408.58
Personal Auto  F                  452.97
               M                  457.01
Special Auto   F                  453.28
               M                  429.53

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [33]:
state_counts = insurance_data['State'].value_counts()
filtered_states = state_counts[state_counts > 500]
filtered_states

State
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [37]:
import numpy as np
grouped_2 = filtered_df_2.groupby(['Education', 'Gender']).agg({'Customer Lifetime Value':['max', 'min', np.median]}).round(2)

grouped_2

/var/folders/h0/4jpjr28x5yj6tn2nqwlp8cn40000gn/T/ipykernel_5180/26963455.py:2: FutureWarning: The provided callable <function median at 0x10d9a5940> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  grouped_2 = filtered_df_2.groupby(['Education', 'Gender']).agg({'Customer Lifetime Value':['max', 'min', np.median]}).round(2)


Customer Lifetime Value                  
                                                max      min   median
Education            Gender                                          
Bachelor             F                     33473.35  2248.45  5246.28
                     M                     24127.50  2300.69  5548.03
College              F                     25807.06  2004.35  4834.71
                     M                     22332.44  2393.92  5989.77
Doctor               F                      6265.34  2395.57  4673.46
                     M                     12731.95  2491.32  5073.28
High School or Below F                     41787.90  2397.04  6503.40
                     M                     12298.69  2227.07  6366.23
Master               F                     19160.99  2574.02  5096.67
                     M                     14435.67  2551.23  8509.85

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9